In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Time-Series-in-IOT'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data = torch.from_numpy(np.array(data['Vehicles'].dropna().tolist())[:5000]).view(1,-1)

In [4]:
data.shape

torch.Size([1, 10000])

In [5]:
data_input = data[:1,:-1].to(device).float()
data_target = data[:1,1:].to(device).float()

In [6]:
data_input

tensor([[15., 13., 10.,  ..., 82., 76., 78.]], device='cuda:0')

In [7]:
data_target

tensor([[13., 10.,  7.,  ..., 76., 78., 79.]], device='cuda:0')

In [8]:
class Model(Module):
    def __init__(self,hidden=256):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden).to(device)
        self.lstm2 = LSTMCell(self.hidden,self.hidden).to(device)
        self.linear = Linear(self.hidden,1).to(device)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [9]:
model = Model().to(device)

In [10]:
criterion = MSELoss()

In [11]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [12]:
epochs = 100

In [13]:
epochs_iter = tqdm(range(epochs))

  0%|                                                   | 0/100 [00:00<?, ?it/s]

In [14]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in epochs_iter:
    def closure():
        optimizer.zero_grad()
        epochs_iter.set_description('predicting')
        preds = model(data_input)
        epochs_iter.set_description('loss')
        loss = criterion(preds,data_target)
        epochs_iter.set_description('loss backward')
        loss.backward()
        epochs_iter.set_description('logging')
        wandb.log({'Loss':loss.item()})
        return loss
    epochs_iter.set_description('step')
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        epochs_iter.set_description('test predicting')
        preds = model(data_input,future)
        epochs_iter.set_description('loss')
        loss = criterion(preds[:,:-future],data_target)
        epochs_iter.set_description('logging')
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].cpu().detach().numpy()
        n = data_input.shape[1]
        epochs_iter.set_description('plotting')
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        epochs_iter.set_description('logging')
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


Problem at: /tmp/ipykernel_374385/535686694.py 1 <module>



CondaError: KeyboardInterrupt



KeyboardInterrupt: 